In [1]:
%load_ext autoreload
%autoreload 2
from dataset import SELMADataset
import open3d as o3d
import seaborn as sns
import copy
import json
from tqdm import trange
from dataset import SELMADataset, get_dataloader
import numpy as np

data_aug = dict(object_noise=dict(
                   num_try=100,
                   translation_std=[0.25, 0.25, 0.25],
                   rot_range=[-0.15707963267, 0.15707963267]
                   ),
               random_flip_ratio=0.5,
               global_rot_scale_trans=dict(
                   rot_range=[-0.78539816, 0.78539816],
                   scale_ratio_range=[0.95, 1.05],
                   translation_std=[0, 0, 0]
                   ), 
               point_range_filter=[-100.0, -100.0, -1, 100.0, 100.0, 3],
               object_range_filter=[-100.0, -100.0, -1, 100.0, 100.0, 3]             
           )

ds = SELMADataset(root_path="../PointPillars/SELMA/CV/dataset/",
             splits_path="./dataset/ImageSets",
             split="all",
             split_extension="txt",
             augment_data=False,
            #  flip=args.random_flip,
            #  gaussian_blur=args.gaussian_blur,
            #  blur_mul=args.blur_mul,
            #  gaussian_noise=args.gaussian_noise,
            #  noise_mul=args.noise_mul,
            #  color_shift=args.color_shift,
            #  color_jitter=args.color_jitter,
            #  cshift_intensity=args.cshift_intensity,
            #  wshift_intensity=args.wshift_intensity,
            sensors=['lidar', 'bbox'],
            #  weather='clear',
            #  time_of_day='sunset',
            sensor_positions=['T'],
            bbox_location="../PointPillars/SELMA/corrected_bbox/",
            n_min=5,
            lidar_data_aug_config=data_aug
            #  class_set='city19',
            #  return_grayscale=args.input_channels==1)
)

def draw_sample(sample):
    pcs = []
    mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(
                            size=2, origin=[0,0,0])
    pcs.append(mesh_frame)

    palette = sns.color_palette("hsv", n_colors=3)

    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(sample['pts'])
    pcs.append(pcd)
        
    for bbox, label in zip(sample['gt_bboxes_3d'], sample['gt_labels']):

        
        
        bb = o3d.geometry.OrientedBoundingBox(np.array([bbox[0],
                                                        bbox[1],
                                                        bbox[2]]),
                                              o3d.geometry.get_rotation_matrix_from_xyz([0.0,0.0,bbox[6]]),
                                              np.array([bbox[3],
                                                        bbox[4],
                                                        bbox[5]]),
                                                        )
        bb.color = palette[label]
        pcs.append(bb)
    o3d.visualization.draw_geometries(pcs)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [8]:
for i in [0, 5000, 10000, 15000, 20000]:
    sample = ds[i]
    print(sample["gt_bboxes_3d"].dtype)
    draw_sample(sample)
    # print(np.min(sample["pts"], axis=0))
    # print(np.max(sample["pts"], axis=0))

float32
float32
float32
float32
float32
